# Word2vec preprocessing

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']

# Skip-Gram

indices_to_words(x_batch) = ['against', 'early', 'working', 'class']

indices_to_words(labels_batch) = ['used', 'working', 'early', 'radicals']]

```

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [99]:
import numpy as np
from itertools import islice

In [100]:
with open('./dataset/corpus', 'rt') as fp:
    corpus = fp.read()

corpus = corpus.strip()
corpus = corpus.split()

In [101]:
class SkipGramBatcher:
    def __init__(self, corpus, window_size=4, batch_size=32):
        '''corpus - list of words'''
        self.corpus = corpus
        self.window_size = window_size
        self.batch_size = batch_size
        self.make_vocab()
        return
    
    def make_vocab(self):
        self.vocab = sorted(set(corpus))
        self.word2index = {w: idx for idx, w in enumerate(self.vocab)}
        self.index2word = {idx: w for idx, w in enumerate(self.vocab)}
        return
    
    def batch_gen(self):
        '''c - corpus, v - vocab ; i - central, j - side'''
        x_batch = np.empty(self.batch_size)
        y_batch = np.empty(self.batch_size)
        curr_idx = 0
        for c_i, w in enumerate(self.corpus):
            v_i = self.word2index[w]
            for side_w in self.corpus[c_i - self.window_size: c_i] \
                          + self.corpus[c_i + 1 : c_i + self.window_size + 1]:
                v_j = self.word2index[side_w]
                x_batch[curr_idx] = v_i
                y_batch[curr_idx] = v_j
                curr_idx += 1
                if curr_idx == self.batch_size:
                    curr_idx = 0
                    yield (x_batch, y_batch)
        if curr_idx != 0:
            yield (x_batch, y_batch)

In [102]:
batcher = SkipGramBatcher(corpus)

In [103]:
max_iter = 100
for x_batch, y_batch in islice(batcher.batch_gen(), max_iter):
    print('x_batch.shape: {}, y_batch.shape: {}'.format(x_batch.shape, y_batch.shape))

x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.shape: (32,), y_batch.shape: (32,)
x_batch.sha